<a href="https://colab.research.google.com/github/quangtrung0220/simpleOcr/blob/main/simpleOcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Import các thư viện cần thiết***

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

import cv2

import numpy as np


### ***Clone git để lấy dữ liệu ảnh train và test***

In [ ]:
# Mount với drive để lấy datasets 
from google.colab import drive
drive.mount('/content/drive')

# Link git lấy dataset 
# !git clone https://github.com/quangtrung0220/SieuAmTim.git
# traindir = "/content/SieuAmTim/DATA_CHAMBER_2021/train"
# testdir = "/content/SieuAmTim/DATA_CHAMBER_2021/test"


Mounted at /content/drive


In [ ]:
traindir = "/content/drive/MyDrive/data"
testdir = "/content/drive/MyDrive/data_test"

### ***Lấy ra các nhãn - classes và chuẩn bị dữ liệu***


---



*   Các nhãn: HighLands, StarBucks, Others, PhucLong
*   KKích cỡ ảnh train và test: 224x224
*   Preprocess: Cân bằng sáng: histogram, Lọc nhiễu: GaussianBlur
*   Data Augmentation: RandomCrop, RandomHorizontalFlip, RandomVerticalFlip
*   Đưa vào batch để tận dụng khả năng xử lý song song của GPU
*   Folder train và test lần lượt trong: traindir và testdir






In [ ]:
# Hàm lấy ra các classes
def get_clases():
  classes = ['HighLands', 'Other', 'PhucLong', 'StarBuck']
  return classes

TrainTest = namedtuple('TrainTest', ['train', 'test'])

# Hàm chuẩn bị dữ liệu
def prepare_data():

  #Augmentation for all
  horizontal = transforms.RandomHorizontalFlip()
  vertical = transforms.RandomVerticalFlip()

  #Image 224x224
  resize_224 = transforms.Resize((224,224))
  crop_224 = transforms.RandomCrop(224, padding=4)

  #To tensor
  tensor = transforms.ToTensor()

  #Raw image
  transform_train_raw224 = transforms.Compose([resize_224, tensor])

  #Augmentation image
  transform_train_aug224 = transforms.Compose([resize_224, crop_224, horizontal, vertical, tensor])

  #Test image
  transform_test_224 = transforms.Compose([resize_224, tensor])

  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train_raw224)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test_224)
  return TrainTest(train=trainset, test=testset)

# Hàm load dữ liệu
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=8, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=8, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

### ***Hàm train và test***


---



*   BatchSize = 8
*   Report sau mỗi 9 epoch 
*   Train: cho đi qua model và tính lỗi bằng loss_func sau đó cập nhật tham số mới
*   Test: khai báo chế độ eval(đánh giá - evaluate) và trả về nhãn dự đoán và nhãn thực tế



In [ ]:
# Data = prepare_data()
# img, label = Data.train[3000]
# print(label)
# plt.show(img)
# print(img)
# s = Data.train.imgs[0]
# print(s)

In [ ]:
# Hàm train 
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true_result = []
  pred_result = []

  # gọi hàm train để model biết mình phải cần huấn luyện 
  model.train()
  running_loss = 0.0
  reporting_steps = 9
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true_result += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred_result += list(predicted.cpu().numpy())
    # Tính lỗi 
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  # return pred_result, true_result

# hàm test 
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  # Gọi hàm eval để model biết đang ở chế độ test 
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

### ***Hàm main***

---


*   model_in = 'vgg16', 'resnet50' hoặc 'desnet': tên mạng Cnn đùng để huấn luyện
*   PATH: đường dẫn lưu trọng số sau khi huấn luyện
*   Đặc trưng ra = 4 vì có 4 lớp
*   Hàm lỗi được sử dụng: Cross-Entropy
*   Hàm tối ưu đước sử dụng: SGD - Stochastic Gradient Descent




In [ ]:
def main(PATH='./model.pth', model_in=''):
  classes = get_clases()
  datasets = prepare_data()
  # img, label = datasets.train[0]
  # plt.imshow(img)
  # print(classes[label], img.size)
  # print('train', len(datasets.train), 'test', len(datasets.test))
  
  loaders = prepare_loader(datasets)

  # images, labels = iter(loaders.train).next()
  # print(images.shape, labels.shape)

  device = torch.device("cuda:0")
  # torch.cuda.empty_cache()

  # model = ResNet50().to(device)
  # model = VGG16().to(device)
  # images, labels = iter(loaders.train).next()
  # outputs = model(images)
  # print(outputs.shape)
  # print(outputs[0])
  # _, predicted = torch.max(outputs, dim=1)
  # print(predicted)
  # imshow(images, labels, predicted, classes)

# Load model 
  if model_in == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=4, bias=True)
    # if torch.cuda.is_available():
    model.to(device=device)
  elif model_in == 'resnet101':
    model = torchvision.models.resnet101(pretrained=False, progress=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=4, bias=True)  
    # if torch.cuda.is_available():
    model.to(device=device)
  elif model_in == 'desnet':
    model = torchvision.models.densenet121(pretrained=False, progress=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=4, bias=True)
    # if torch.cuda.is_available():
    model.to(device=device)
  else:
    pass  

  # array_accuracy = []

  loss_func = nn.CrossEntropyLoss()

  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(50):
    print(f'Epoch {epoch} report: ')
    
    # pred_result, true_result = 
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    # print('Train report: ')
    # print(classification_report(true_result, pred_result, target_names=classes))

    print('Test report: ')
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)

    # accuracy = (ytrue_test==ypred_test).sum() / len(ytrue_test)
    # array_accuracy.append(accuracy)

    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

  return model

model = main(PATH="./densenet121.pth", model_in='desnet')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 report: 
Epoch 0 step 8 ave_loss 1.6465
Epoch 0 step 17 ave_loss 1.5482
Epoch 0 step 26 ave_loss 1.1201
Epoch 0 step 35 ave_loss 1.1188
Epoch 0 step 44 ave_loss 1.6656
Epoch 0 step 53 ave_loss 1.4180
Test report: 
              precision    recall  f1-score   support

   HighLands       0.70      0.12      0.20        59
       Other       0.57      0.38      0.46        78
    PhucLong       0.16      0.45      0.23        31
    StarBuck       0.11      0.17      0.13        30

    accuracy                           0.28       198
   macro avg       0.38      0.28      0.26       198
weighted avg       0.47      0.28      0.30       198

Epoch 1 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 step 8 ave_loss 1.4958
Epoch 1 step 17 ave_loss 2.2915
Epoch 1 step 26 ave_loss 1.8039
Epoch 1 step 35 ave_loss 1.7962
Epoch 1 step 44 ave_loss 1.3125
Epoch 1 step 53 ave_loss 1.3917
Test report: 
              precision    recall  f1-score   support

   HighLands       0.00      0.00      0.00        59
       Other       0.49      0.99      0.65        78
    PhucLong       0.00      0.00      0.00        31
    StarBuck       0.44      0.37      0.40        30

    accuracy                           0.44       198
   macro avg       0.23      0.34      0.26       198
weighted avg       0.26      0.44      0.32       198

Epoch 2 report: 


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.

...

Epoch 47 step 8 ave_loss 0.0076
Epoch 47 step 17 ave_loss 0.0029
Epoch 47 step 26 ave_loss 0.0047
Epoch 47 step 35 ave_loss 0.0011
Epoch 47 step 44 ave_loss 0.0028
Epoch 47 step 53 ave_loss 0.0016
Test report: 
              precision    recall  f1-score   support

   HighLands       0.98      0.98      0.98        59
       Other       0.96      1.00      0.98        78
    PhucLong       1.00      0.94      0.97        31
    StarBuck       0.97      0.93      0.95        30

    accuracy                           0.97       198
   macro avg       0.98      0.96      0.97       198
weighted avg       0.98      0.97      0.97       198

Epoch 48 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 48 step 8 ave_loss 0.0077
Epoch 48 step 17 ave_loss 0.0004
Epoch 48 step 26 ave_loss 0.0006
Epoch 48 step 35 ave_loss 0.0045
Epoch 48 step 44 ave_loss 0.0031
Epoch 48 step 53 ave_loss 0.0105
Test report: 
              precision    recall  f1-score   support

   HighLands       1.00      0.98      0.99        59
       Other       0.96      1.00      0.98        78
    PhucLong       1.00      0.94      0.97        31
    StarBuck       0.97      0.97      0.97        30

    accuracy                           0.98       198
   macro avg       0.98      0.97      0.98       198
weighted avg       0.98      0.98      0.98       198

Epoch 49 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 49 step 8 ave_loss 0.0028
Epoch 49 step 17 ave_loss 0.0018
Epoch 49 step 26 ave_loss 0.0117
Epoch 49 step 35 ave_loss 0.0019
Epoch 49 step 44 ave_loss 0.0024
Epoch 49 step 53 ave_loss 0.0049
Test report: 
              precision    recall  f1-score   support

   HighLands       0.98      1.00      0.99        59
       Other       0.96      1.00      0.98        78
    PhucLong       1.00      0.94      0.97        31
    StarBuck       1.00      0.93      0.97        30

    accuracy                           0.98       198
   macro avg       0.99      0.97      0.98       198
weighted avg       0.98      0.98      0.98       198



### ***Test Code ***

In [ ]:
print(torchvision.models.densenet121())

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu